## Graph data structures

Suppose there are $N$ vertices and $E$ edges in graph $G$.

| Operation                         | Adj. Matrix | Adj. List        | Edge List |
|-----------------------------------|-------------|------------------|-----------|
| Enumerate (all neighbours of $v$) | $O(N)$      | $O(\frac{E}{N})$ | $O(E)$    |
| Find (if $(u,v)$ are connected)   | $O(1)$      | $O(\frac{E}{N})$ | $O(E)$    |
| Insert (an edge)                  | $O(1)$      | $O(1)$           | $O(1)$    |
| Delete (an edge)                  | $O(1)$      | $O(\frac{E}{N})$ | $O(1)$    |
| Storage                           | $O(N^2)$    | $O(E)$           | $O(E)$    |

Note that $\frac{E}{N}$ is actually the average degree $k$ in graph $G$.

## Exercise

1. Write a function `enumerate_matrix` that takes full adjacency matrix (as a $N \times N$ matrix in R, or as a $N \times N$ `numpy` matrix in python) and an integer vertex number `i` as input, and returns a list (or vector in the case of R) of the vertex numbers of the neighboring vertices
2. Write a function `enumerate_adj_list` that takes an adjacency list data structure (as a list of vectors in R, and as a list of lists in Python) and an integer vertex number `i` as input, and returns a list (or vector in the case of R) of the vertex numbers of the neighboring vertices
3. Write a function `enumerate_edge_list` that takes an edge-list data structure (as a $N \times 2$ data frame in R, and as a $N \times 2$ `numpy.ndarray` in python) and an integer vertex number `i` as input, and returns a list (or vector in the case of R) of the vertex numbers of the neighboring vertices
4. Write a function `dosim` that takes a single parameter $n$ (number of vertices) that does the following procedure 10 times internally, taking the resulting three-vector from each time and stacking them and averaging the results column-wise:
    - generate a Barabasi-Albert graph with $n$ vertices and $5 \times n$ edges
    - obtain the adjacency matrix for the graph, and obtain the overall running time for running `enumerate_matrix` on this object 1,000 times for each value of n as the `i` parameter
    - obtain the adjacency list for the graph, and obtain the overall running time for running `enumerate_adj_list` on this object 1,000 times for each value of n as the `i` parameter
    - obtain the edge-list for the graph, and obtain the overall running time for running `enumerate_edge_list` on this object 1,000 times for each value of n as the “i” parameter
    - resulting three-vector is the running time for each of the above three steps


# Class Session 4 Exercise:
## Comparing asymptotic running time for enumerating vertex neighbors

We will measure the running time for enumerating the neighbor vertices for three different data structures for representing the graph:
    adjacency matrix
    adjacency list
    edge list
    
First, we import all the Python modules that we will need for this exercise:

In [2]:
import numpy as np
import igraph
import timeit
import itertools

Define a function that enumerates the neighbors of a vertex i, when the 
graph is stored in adjacency matrix format

In [33]:
def enumerate_matrix(gmat, i):
    # np.nonzero(matrix) 返回 (array([row, ...]), array([col, ...]))
    return np.nonzero(gmat[i,:])[1].tolist()

# make a random undirected graph with 4 vertices and fixed (average) vertex degree = 3
g = igraph.Graph.Barabasi(4, 2)

print(g.get_adjacency())
print(type(g.get_adjacency()))
print(type(g.get_adjacency()[1,:]))
print("=====")
print(g.get_adjacency().data)
print(type(g.get_adjacency().data))
print("=====")
print(np.matrix(g.get_adjacency().data))
print(np.nonzero(np.matrix(g.get_adjacency().data)[1,:]))
print("=====")

print("N.B Difference!")
gmat1 = g.get_adjacency()
gmat2 = np.matrix(g.get_adjacency().data)
print(np.nonzero(gmat1[1,:]))  # 如果 enumerate_matrix 的参数是这个 matrix，应该返回 np.nonzero(gmat[i,:])[0]
print(np.nonzero(gmat2[1,:]))

print(enumerate_matrix(gmat2, 1))

[[0, 1, 1, 1]
 [1, 0, 1, 1]
 [1, 1, 0, 0]
 [1, 1, 0, 0]]
<class 'igraph.datatypes.Matrix'>
<class 'list'>
=====
[[0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 0, 0], [1, 1, 0, 0]]
<class 'list'>
=====
[[0 1 1 1]
 [1 0 1 1]
 [1 1 0 0]
 [1 1 0 0]]
(array([0, 0, 0]), array([0, 2, 3]))
=====
N.B Difference!
(array([0, 2, 3]),)
(array([0, 0, 0]), array([0, 2, 3]))
[0, 2, 3]


Define a function that enumerates the neighbors of a vertex i, when the 
graph is stored in adjacency list format

In [16]:
def enumerate_adj_list(adj_list, i):
    return adj_list[i]

g_al = g.get_adjlist()

print(g_al)
print(enumerate_adj_list(g_al, 1))

[[1, 2, 3], [0, 2, 3], [0, 1], [0, 1]]
[0, 2, 3]


Define a function that enumerates the neighbors of a vertex i, when the 
graph is stored in edge-list format

In [36]:
def enumerate_edge_list(edge_list, i):
    inds1 = np.where(edge_list[:,0] == i)[0]
    elems1 = edge_list[inds1, 1].tolist()
    inds2 = np.where(edge_list[:,1] == i)[0]
    elems2 = edge_list[inds2, 0].tolist()
    return np.unique(elems1 + elems2).tolist()

# g_el = [edge.tuple for edge in g.es]  # Equivlent!
g_el = g.get_edgelist() 

print(g_el)  # a list of tuples
print(np.array(g_el))  # a nx2 array
print(enumerate_edge_list(np.array(g_el), 1))

[(0, 1), (0, 2), (1, 2), (1, 3), (0, 3)]
[[0 1]
 [0 2]
 [1 2]
 [1 3]
 [0 3]]
[0, 2, 3]


This next function is the simulation funtion.  "n" is the number of vertices.
It returns a length-three list containing the average running time for enumerating the neighbor vertices of a vertex in the graph. 

In [25]:
def do_sim(n):
    """
        n - number of vertices
    """

    retlist = []
    
    nrep = 10  # 产生 10 张图
    nsubrep = 10  # 在每张图上做 10 次操作，每次操作都从 0 遍历到 N-1
    
    # this is (sort of) a Python way of doing the R function "replicate":
    for _ in itertools.repeat(None, nrep):
        
        # make a random undirected graph with fixed (average) vertex degree = 5
        g = igraph.Graph.Barabasi(n, 5)
        
        # get the graph in three different representations
        g_matrix = np.matrix(g.get_adjacency().data)
        g_adj_list = g.get_adjlist()
        g_edge_list = np.array(g.get_edgelist())
    
        start_time = timeit.default_timer()
    
        for _ in itertools.repeat(None, nsubrep):
            for i in range(0, n):
                enumerate_matrix(g_matrix, i)
    
        matrix_elapsed = timeit.default_timer() - start_time
        
        start_time = timeit.default_timer()
        
        for _ in itertools.repeat(None, nsubrep):
             for i in range(0, n):
                enumerate_adj_list(g_adj_list, i)        
        
        adjlist_elapsed = timeit.default_timer() - start_time
        
        start_time = timeit.default_timer()
        
        for _ in itertools.repeat(None, nsubrep):
             for i in range(0, n):
                enumerate_edge_list(g_edge_list, i)        
        
        edgelist_elapsed = timeit.default_timer() - start_time
        
        retlist.append([matrix_elapsed, adjlist_elapsed, edgelist_elapsed])

        # average over replicates and then
        # divide by n so that the running time results are on a per-vertex basis
    return np.mean(np.array(retlist), axis=0)/n

A simulation with 1000 vertices clearly shows that adjacency list is fastest:
(I multiply by 1000 just so the results are in ms.)

In [23]:
do_sim(1000)*1000

array([ 0.14328251,  0.00177659,  0.3966931 ])

We see the expected behavior, with the running time for the adjacency-matrix and edge-list formats going up when we increase "n", but there is hardly any change in the running time for the graph stored in adjacency list format:

In [24]:
do_sim(2000)*1000

array([ 0.20474197,  0.00181276,  0.53985972])